In [7]:
import numpy as np
import os
import random
import pandas as pd
import datetime
import math
from math import e
import datetime
from scipy.interpolate import interp1d
import time, datetime
from pandas.core.frame import DataFrame
from db_models import GmsMonitor

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [8]:
def pd_aggregation(data):
    dense_series_1s = data.resample('1S').interpolate("linear")
    series_1m = dense_series_1s.groupby(pd.Grouper(freq='5Min')).aggregate(np.mean)
    return series_1m

In [9]:
def mongodb_connect():
    import mongoengine
    mongoengine
    mongoengine.connect('nfca_db', host='dgx.server.ustb-ai3d.cn', port=27017, username='nfca', password='nfca')

mongodb_connect()

In [10]:
def queryset2df(query_data):
    """
    queryset转dataframe
    :return:
    """
    dic = {"time":[], "value":[]}
    for q in query_data:
        dic["time"].append(q['time'])
        dic["value"].append(q['Monitoring_value'])
    # 存在数据库多条目同一时间同一值、多条目同一时间不同值的情况，仅保留首条数据
    df = pd.DataFrame(dic).drop_duplicates(subset=['time'])
    return df

In [14]:
df = pd.read_csv('30s_label1_mesto.csv')
df

,time,feed_c,out_c,feed_f,out_f,pressure,fill_round,label
0,2021/5/29 11:49:00,19.117509,64.373919,581.728474,47.624088,42.420909,0,1
1,2021/5/29 11:49:30,18.902742,65.435274,584.560961,47.756499,42.243740,0,1
2,2021/5/29 11:50:00,18.687975,66.496628,587.393447,47.888910,42.066571,0,1
3,2021/5/29 11:50:30,18.434042,66.929339,587.165821,51.951373,41.153174,0,1
4,2021/5/29 11:51:00,18.180109,67.362050,586.938195,56.013837,40.239776,0,1
...,...,...,...,...,...,...,...,...
127485,2022/5/7 10:08:30,29.186452,59.767718,626.590778,140.018833,51.829096,188,1
127486,2022/5/7 10:09:00,28.395807,59.835951,631.875293,138.250821,51.930065,188,1
127487,2022/5/7 10:09:30,28.569112,59.920804,665.998777,138.257720,51.908868,188,1
127488,2022/5/7 10:10:00,28.742416,60.005657,700.122260,138.264618,51.887671,188,1


In [13]:
df['time'] = pd.to_datetime(df['time'])
df

,time,feed_c,out_c,feed_f,out_f,pressure,fill_round,label
0,2021-05-29 11:49:00,19.117509,64.373919,581.728474,47.624088,42.420909,0,1
1,2021-05-29 11:49:30,18.902742,65.435274,584.560961,47.756499,42.243740,0,1
2,2021-05-29 11:50:00,18.687975,66.496628,587.393447,47.888910,42.066571,0,1
3,2021-05-29 11:50:30,18.434042,66.929339,587.165821,51.951373,41.153174,0,1
4,2021-05-29 11:51:00,18.180109,67.362050,586.938195,56.013837,40.239776,0,1
...,...,...,...,...,...,...,...,...
127485,2022-05-07 10:08:30,29.186452,59.767718,626.590778,140.018833,51.829096,188,1
127486,2022-05-07 10:09:00,28.395807,59.835951,631.875293,138.250821,51.930065,188,1
127487,2022-05-07 10:09:30,28.569112,59.920804,665.998777,138.257720,51.908868,188,1
127488,2022-05-07 10:10:00,28.742416,60.005657,700.122260,138.264618,51.887671,188,1


In [31]:
df = df.set_index(['time'], drop=True)

In [32]:
df.drop(columns = ['fill_round'],inplace = True)
df_1 = df.resample('5Min').interpolate("zero")
df_1 = df_1.drop(df_1.index[0])

In [33]:
df_1.to_csv("df_test.csv", index=True)

In [34]:
df_1

,feed_c,out_c,feed_f,out_f,pressure,label
time,,,,,,
2021-05-29 11:50:00,18.687975,66.496628,587.393447,47.888910,42.066571,1.0
2021-05-29 11:55:00,18.514265,68.554742,594.701354,54.951627,36.228900,1.0
2021-05-29 12:00:00,20.828083,65.206491,600.521913,137.307769,49.096255,1.0
2021-05-29 12:05:00,18.641244,65.202654,619.654978,133.066213,42.642862,1.0
2021-05-29 12:10:00,21.932207,65.288382,631.921162,123.592218,43.254578,1.0
...,...,...,...,...,...,...
2022-05-07 09:50:00,33.020739,59.366599,601.963274,146.562356,51.578216,1.0
2022-05-07 09:55:00,29.091299,60.463238,679.524404,136.653616,50.474077,1.0
2022-05-07 10:00:00,26.692466,59.308527,601.825201,157.848585,51.421173,1.0


In [35]:
out_f_setting_query = (GmsMonitor.objects.filter(
    time__gte=df_1.index[0],
    time__lt=df_1.index[-1],
    point_id=73
).order_by("time"))
out_f_setting = []
for query in out_f_setting_query:
    out_f_setting.append(query.to_mongo().to_dict())
out_f_setting = queryset2df(out_f_setting).set_index(['time'], drop=False)

# 最后时间的前一个时间
past_set_point =  GmsMonitor.objects.filter(
    time__lt=df_1.index[0],
    point_id=73
).order_by("-time").limit(1)[0].to_mongo().to_dict()
past_time = past_set_point['time']
past_set_value = past_set_point['Monitoring_value']
out_f_setting.loc[past_time] = [past_time, past_set_value]

# 最新时间
last_set_point = GmsMonitor.objects.filter(
    point_id=73
).order_by("-time").limit(1)[0].to_mongo().to_dict()
last_time = datetime.datetime.now()
last_set_value = last_set_point['Monitoring_value']
out_f_setting.loc[last_time] = [last_time, last_set_value]

# 0阶插值\pad上采样
out_f_setting = out_f_setting.sort_index()
out_f_setting = out_f_setting.resample('1S').pad().groupby(pd.Grouper(freq='5Min')).aggregate(np.mean)


out_f_setting = out_f_setting.loc[df_1.index[0]: df_1.index[-1]]   # 和out_c区间保持一致
out_f_setting.rename(columns={"value":"out_f_setting"}, inplace=True)

In [36]:
out_f_setting
df_1 = pd.concat([df_1, out_f_setting],axis=1)

In [37]:
out_c_setting_query = (GmsMonitor.objects.filter(
    time__gte=df_1.index[0],
    time__lt=df_1.index[-1],
    point_id=70
).order_by("time"))
out_c_setting = []
for query in out_c_setting_query:
    out_c_setting.append(query.to_mongo().to_dict())
out_c_setting = queryset2df(out_c_setting).set_index(['time'], drop=False)

# 最后时间的前一个时间
past_set_point =  GmsMonitor.objects.filter(
    time__lt=df_1.index[0],
    point_id=70
).order_by("-time").limit(1)[0].to_mongo().to_dict()
past_time = past_set_point['time']
past_set_value = past_set_point['Monitoring_value']
out_c_setting.loc[past_time] = [past_time, past_set_value]

# 最新时间
last_set_point = GmsMonitor.objects.filter(
    point_id=73
).order_by("-time").limit(1)[0].to_mongo().to_dict()
last_time = datetime.datetime.now()
last_set_value = last_set_point['Monitoring_value']
out_c_setting.loc[last_time] = [last_time, last_set_value]

# 0阶插值\pad上采样
out_c_setting = out_c_setting.sort_index()
out_c_setting = out_c_setting.resample('1S').pad().groupby(pd.Grouper(freq='5Min')).aggregate(np.mean)


out_c_setting = out_c_setting.loc[df_1.index[0]: df_1.index[-1]]   # 和out_c区间保持一致
out_c_setting.rename(columns={"value":"out_c_setting"}, inplace=True)

In [38]:
out_c_setting
df_1 = pd.concat([df_1, out_c_setting],axis=1)

In [39]:
df_1

,feed_c,out_c,feed_f,out_f,pressure,label,out_f_setting,out_c_setting
time,,,,,,,,
2021-05-29 11:50:00,18.687975,66.496628,587.393447,47.888910,42.066571,1.0,133.133333,52.0
2021-05-29 11:55:00,18.514265,68.554742,594.701354,54.951627,36.228900,1.0,141.116667,52.0
2021-05-29 12:00:00,20.828083,65.206491,600.521913,137.307769,49.096255,1.0,135.000000,52.0
2021-05-29 12:05:00,18.641244,65.202654,619.654978,133.066213,42.642862,1.0,131.366667,52.0
2021-05-29 12:10:00,21.932207,65.288382,631.921162,123.592218,43.254578,1.0,130.000000,52.0
...,...,...,...,...,...,...,...,...
2022-05-07 09:50:00,33.020739,59.366599,601.963274,146.562356,51.578216,1.0,140.399994,62.0
2022-05-07 09:55:00,29.091299,60.463238,679.524404,136.653616,50.474077,1.0,140.399994,62.0
2022-05-07 10:00:00,26.692466,59.308527,601.825201,157.848585,51.421173,1.0,140.399994,62.0


In [62]:
df_2 = df_1
len(df_2)

98765

In [58]:
# df_2 = df_2.drop(df_2.index[1])

In [59]:
# df_2

In [67]:
# i = 0
# while i < len(df_2):
#     if i == 0: 
#         i = i + 1
#     elif i == len(df_2):
#         print("Data delete over!")
#         break
#     elif df_2['pressure'][i] == df_2['pressure'][i-1]:
#         df_2 = df_2.drop(df_2.index[i])
#         print("Empty time_range", i)
#     elif df_2['label'][i] != 1:
#         df_2 = df_2.drop(df_2.index[i])
#         print("label error", i)
#     else:
#         i = i + 1

In [91]:
df_2

,feed_c,out_c,feed_f,out_f,pressure,out_f_setting,out_c_setting,reward,error,sum_error,done
time,,,,,,,,,,,
2021-05-29 11:50:00,18.687975,66.496628,587.393447,47.888910,42.066571,133.133333,52.0,-210.152231,-14.496628,-14.496628,0
2021-05-29 11:55:00,18.514265,68.554742,594.701354,54.951627,36.228900,141.116667,52.0,-274.059494,-16.554742,-31.051371,0
2021-05-29 12:00:00,20.828083,65.206491,600.521913,137.307769,49.096255,135.000000,52.0,-174.411393,-13.206491,-44.257861,0
2021-05-29 12:05:00,18.641244,65.202654,619.654978,133.066213,42.642862,131.366667,52.0,-174.310072,-13.202654,-57.460515,0
2021-05-29 12:10:00,21.932207,65.288382,631.921162,123.592218,43.254578,130.000000,52.0,-176.581087,-13.288382,-70.748897,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-07 09:50:00,33.020739,59.366599,601.963274,146.562356,51.578216,140.399994,62.0,-6.934803,2.633401,8472.714631,0
2022-05-07 09:55:00,29.091299,60.463238,679.524404,136.653616,50.474077,140.399994,62.0,-2.361637,1.536762,8474.251393,0
2022-05-07 10:00:00,26.692466,59.308527,601.825201,157.848585,51.421173,140.399994,62.0,-7.244026,2.691473,8476.942866,0


In [66]:
df_2.to_csv("df_test.csv", index=True)

In [92]:
df_3 = df_2

In [90]:
df_3.drop(columns = ['label'],inplace = True)

KeyError: "['label'] not found in axis"

In [87]:
df_3

,feed_c,out_c,feed_f,out_f,pressure,out_f_setting,out_c_setting,reward,error,sum_error,done
time,,,,,,,,,,,
2021-05-29 11:50:00,18.687975,66.496628,587.393447,47.888910,42.066571,133.133333,52.0,-210.152231,-14.496628,-14.496628,0
2021-05-29 11:55:00,18.514265,68.554742,594.701354,54.951627,36.228900,141.116667,52.0,-274.059494,-16.554742,-31.051371,0
2021-05-29 12:00:00,20.828083,65.206491,600.521913,137.307769,49.096255,135.000000,52.0,-174.411393,-13.206491,-44.257861,0
2021-05-29 12:05:00,18.641244,65.202654,619.654978,133.066213,42.642862,131.366667,52.0,-174.310072,-13.202654,-57.460515,0
2021-05-29 12:10:00,21.932207,65.288382,631.921162,123.592218,43.254578,130.000000,52.0,-176.581087,-13.288382,-70.748897,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-07 09:50:00,33.020739,59.366599,601.963274,146.562356,51.578216,140.399994,62.0,-6.934803,2.633401,8472.714631,0
2022-05-07 09:55:00,29.091299,60.463238,679.524404,136.653616,50.474077,140.399994,62.0,-2.361637,1.536762,8474.251393,0
2022-05-07 10:00:00,26.692466,59.308527,601.825201,157.848585,51.421173,140.399994,62.0,-7.244026,2.691473,8476.942866,0


In [88]:
df_3['reward'] = - pow((df_3['out_c'] - df_3['out_c_setting']), 2)

In [75]:
df_3['error'] = df_3['out_c_setting'] - df_3['out_c']

In [77]:
df_3['sum_error'] = df_3['error']

In [80]:
df_3['done'] = 0

In [81]:
df_3

,feed_c,out_c,feed_f,out_f,pressure,out_f_setting,out_c_setting,reward,error,sum_error,done
time,,,,,,,,,,,
2021-05-29 11:50:00,18.687975,66.496628,587.393447,47.888910,42.066571,133.133333,52.0,-210.152231,-14.496628,-14.496628,0
2021-05-29 11:55:00,18.514265,68.554742,594.701354,54.951627,36.228900,141.116667,52.0,-274.059494,-16.554742,-31.051371,0
2021-05-29 12:00:00,20.828083,65.206491,600.521913,137.307769,49.096255,135.000000,52.0,-174.411393,-13.206491,-44.257861,0
2021-05-29 12:05:00,18.641244,65.202654,619.654978,133.066213,42.642862,131.366667,52.0,-174.310072,-13.202654,-57.460515,0
2021-05-29 12:10:00,21.932207,65.288382,631.921162,123.592218,43.254578,130.000000,52.0,-176.581087,-13.288382,-70.748897,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-07 09:50:00,33.020739,59.366599,601.963274,146.562356,51.578216,140.399994,62.0,-6.934803,2.633401,8472.714631,0
2022-05-07 09:55:00,29.091299,60.463238,679.524404,136.653616,50.474077,140.399994,62.0,-2.361637,1.536762,8474.251393,0
2022-05-07 10:00:00,26.692466,59.308527,601.825201,157.848585,51.421173,140.399994,62.0,-7.244026,2.691473,8476.942866,0


In [96]:
for i in range(len(df_3) - 1):
    df_3['sum_error'][i+1] = df_3['error'][i+1] + 0.5 * df_3['sum_error'][i]

<ipython-input-96-f3982dc183ad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['sum_error'][i+1] = df_3['error'][i+1] + 0.5 * df_3['sum_error'][i]


In [97]:
df_3

,feed_c,out_c,feed_f,out_f,pressure,out_f_setting,out_c_setting,reward,error,sum_error,done
time,,,,,,,,,,,
2021-05-29 11:50:00,18.687975,66.496628,587.393447,47.888910,42.066571,133.133333,52.0,-210.152231,-14.496628,-14.496628,0
2021-05-29 11:55:00,18.514265,68.554742,594.701354,54.951627,36.228900,141.116667,52.0,-274.059494,-16.554742,-23.803056,0
2021-05-29 12:00:00,20.828083,65.206491,600.521913,137.307769,49.096255,135.000000,52.0,-174.411393,-13.206491,-25.108019,0
2021-05-29 12:05:00,18.641244,65.202654,619.654978,133.066213,42.642862,131.366667,52.0,-174.310072,-13.202654,-25.756663,0
2021-05-29 12:10:00,21.932207,65.288382,631.921162,123.592218,43.254578,130.000000,52.0,-176.581087,-13.288382,-26.166713,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-07 09:50:00,33.020739,59.366599,601.963274,146.562356,51.578216,140.399994,62.0,-6.934803,2.633401,4.412321,0
2022-05-07 09:55:00,29.091299,60.463238,679.524404,136.653616,50.474077,140.399994,62.0,-2.361637,1.536762,3.742922,0
2022-05-07 10:00:00,26.692466,59.308527,601.825201,157.848585,51.421173,140.399994,62.0,-7.244026,2.691473,4.562934,0


In [98]:
df_3.to_csv("df_thickener1_20220522.csv", index=True)